# Cleaning the Data

## Data Prep 

### Load Packages

In [1]:
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
from IPython.display import Image
from ipywidgets import interact, fixed

In [ ]:
import dask.dataframe as dd

In [2]:
df = pd.read_csv("Data/2017.csv")
df.head()

,Data,Unidade,Grupo EFR,Grupo Rúbrica,Tipo Rúbrica,Data Nascimento,Sexo,hashed_ID_Único
0,14/09/2017,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
1,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
2,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
3,18/12/2017,CCTV,ADSE,SERVIÇOS ESPECIAIS UROLOGIA,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
4,24/04/2017,CCSJM,PARTICULARES,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3


### Manipulate Columns

In [3]:
#Translate column names into english
data = df.rename(columns={
    "hashed_ID_Único":"UniqueID",
    "Data":"Date", 
    "Unidade":"Facility",
    "Grupo EFR":"Payer",
    "Grupo Rúbrica":"SpecificService", 
    "Tipo Rúbrica":"CategoryofService",
    "Sexo":"Sex",
    "Data Nascimento":"BirthYear",
})

In [4]:
data.head()

,Date,Facility,Payer,SpecificService,CategoryofService,BirthYear,Sex,UniqueID
0,14/09/2017,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
1,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
2,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
3,18/12/2017,CCTV,ADSE,SERVIÇOS ESPECIAIS UROLOGIA,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
4,24/04/2017,CCSJM,PARTICULARES,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3


In [6]:
import datetime
data["Date"] = pd.to_datetime(data.Date, errors="coerce") #converting to date time 
data["Age"] = data["Date"].dt.year - data["BirthYear"] # Calculate Approximate Age

In [7]:
# Delete Uneeded Columns
del data["BirthYear"]
del data["Date"]

data.head(2)

,Facility,Payer,SpecificService,CategoryofService,Sex,UniqueID,Age
0,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0
1,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0


### Create a customer dataframe 

In [8]:
patient_visits = data.UniqueID.value_counts(sort=False)

In [9]:
customers = patient_visits.rename_axis('UniqueID').reset_index(name='FrequencyofVisits')

In [10]:
customers.head()

,UniqueID,FrequencyofVisits
0,7a1e021f929e5685566a6537f49ad81a,2
1,55a20d53321a1400be757a8ce0b4d804,2
2,7873ceffa470b24cef8bfee10f921e2a,3
3,153db10d47628faeb6a9c8b7bbc9610b,7
4,04eb1c004ba7da3f08c285e082cd6012,3


In [11]:
customers[["Age","Sex"]]= data[["Age","Sex"]]

In [12]:
customers.head()

,UniqueID,FrequencyofVisits,Age,Sex
0,7a1e021f929e5685566a6537f49ad81a,2,37.0,Feminino
1,55a20d53321a1400be757a8ce0b4d804,2,37.0,Feminino
2,7873ceffa470b24cef8bfee10f921e2a,3,37.0,Feminino
3,153db10d47628faeb6a9c8b7bbc9610b,7,37.0,Feminino
4,04eb1c004ba7da3f08c285e082cd6012,3,37.0,Feminino


### Variable Grouping by Age

In [13]:
data['Age_Group']= 0

In [14]:
data['Age_Group'][(data["Age"]<18) & (data["Age"]>=5)] = "Child"
data['Age_Group'][(data["Age"]>=18) & (data["Age"]<44)] = "YoungAdult"
data['Age_Group'][(data["Age"]>=44) & (data["Age"]<64)] = "Adult"
data['Age_Group'][(data["Age"]>=64) & (data["Age"]<81)] = "Senior"
data['Age_Group'][data["Age"]>=81] = "Elderly"

In [15]:
data.head()

,Facility,Payer,SpecificService,CategoryofService,Sex,UniqueID,Age,Age_Group
0,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,YoungAdult
1,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,YoungAdult
2,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,YoungAdult
3,CCTV,ADSE,SERVIÇOS ESPECIAIS UROLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,YoungAdult
4,CCSJM,PARTICULARES,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,YoungAdult


### Data Cleaning

#### Check for missing values 

In [16]:
n_records = len(data)
def missing_values_df(df):
    for column in df:
        print("{} | {} | {}".format(
            column, len(df[df[column].isnull()]) / (1.0*n_records), df[column].dtype
        ))

missing_values_df(data)

Facility | 0.0 | object
Payer | 9.345411715966924e-05 | object
SpecificService | 0.005829963284735586 | object
CategoryofService | 0.005829963284735586 | object
Sex | 0.002336513503076198 | object
UniqueID | 0.0 | object
Age | 2.7137020274886775e-05 | float64
Age_Group | 0.0 | object


In [34]:
null_data = data[data.isnull().any(axis=1)] #looking to see if the missing values where consistently missing for a single cutsomer
null_data.head(2)

,Facility,Payer,SpecificService,CategoryofService,Sex,UniqueID,Age,Age_Group


In [22]:
age_median = data.Age.median() 

data.Payer = data.Payer.fillna("Missing")
data.SpecificService = data.SpecificService.fillna("Missing")
data.CategoryofService = data.CategoryofService.fillna("Missing")
data.Sex = data.Sex.fillna("Missing")
data.Age = data.Age.fillna(age_median)

#replace missing strings with the string "Missing" and the missing age was replaced with the median

In [23]:
missing_values_df(data) #no more missing values

Facility | 0.0 | object
Payer | 0.0 | object
SpecificService | 0.0 | object
CategoryofService | 0.0 | object
Sex | 0.0 | object
UniqueID | 0.0 | object
Age | 0.0 | float64
Age_Group | 0.0 | object


In [26]:
data = data[data.UniqueID != '6bb61e3b7bce0931da574d19d1d82c88'] #deleting a virtual customer from the dataset

### Cleaning Customer df

In [29]:
missing_values_df(customers)

UniqueID | 0.0 | object
FrequencyofVisits | 0.0 | int64
Age | 2.7137020274886775e-05 | float64
Sex | 0.00029577746358781917 | object


In [30]:
customers.Sex = customers.Sex.fillna("Missing")
customers.Age = customers.Age.fillna(age_median)

In [31]:
missing_values_df(customers)

UniqueID | 0.0 | object
FrequencyofVisits | 0.0 | int64
Age | 0.0 | float64
Sex | 0.0 | object


### Export Dataset
Use pickle format so that df reads into notebook faster 

In [32]:
# This is the long format, the transaction table 
data.to_pickle("2017_data.pkl")

In [33]:
# This is the wide format, the customer table 
customers.to_pickle("customer17_data.pkl")